# Contact-based analysis of per-residue local energetic frustration dynamics in fold-switching simulations using the FrustratometeR

This Colab Notebook enables the use of a windowing method for determining which residues exhibit significant changes in local frustration density of contacts around 5 Å, using both contact-based mutational and configurational frustration for the analysis.

This analysis might take a while on Google Colab (~2 frames/min), so it is mainly provided as an example.

**REQUIREMENTS**
- Refolding simulations saved as individual frames in PDB format, numbered from 0 to N, where N is the total of frames
- Upload these PDB files into a folder on Google Colab (e.g. *input*)

In [ ]:
#@title 1. Installing FrustratometeR
import os
print("Installing FrustratometeR")
os.system("pip install udocker")
os.system("udocker --allow-root install")
os.system("udocker --allow-root pull marifrei/frustrar")
print("Installing additional dependencies")
import numpy as np
import sys
print("Installation done!")

In [ ]:
#@title 2. (Optional) Download a representative trajectory as multiple PDB files
import os
import tarfile
print("Downloading short trajectory (400 frames)")
os.system("curl -L -o input.tar.gz https://zenodo.org/records/12594323/files/input_one_fs.tar.gz?download=1")
os.system("tar zxf input.tar.gz")
print("Download completed")


In [ ]:
#@title 3. Set up analysis parameters
#@markdown  - Indicate the input folder where the PDBs of your trajectory are located
InputFolder = 'input_one_fs/' #@param {type:"string"}

# This will be the folder inside the FrustratometeR docker
InputRoot = "/root/"+InputFolder

#@markdown - Indicate the prefix name of the PDB files (e.g. "pdb" for pdb1.pdb, pdb2.pdb...)
prefix = 'pdb' #@param {type:"string"}

#@markdown  - Indicate the output folder for the FrustratometeR analysis
OutputFolder = 'output/' #@param {type:"string"}
# We will create the outputFolder
os.mkdir("/content/"+OutputFolder)
# This will be the folder inside the FrustratometeR docker
OutputRoot = "/root/"+OutputFolder

#@markdown  - Indicate the number of residues in your protein
Nres = 162 #@param {type:"raw"}

#@markdown  - Indicate if the start frame of your simulation is 0 or 1
Start = 0 #@param {type:"raw"}

In [ ]:
#@title 4. Run Mutational and Configurational Frustration analysis for all frames

# Generate a R script for Frustration Analysis
out_r=open('r_frustration.R','w')

# Determining the number of frames
os.system('cd '+InputFolder+';ls *.pdb | wc -l > aux')
aux=open(InputFolder+'aux')
laux=aux.readline()
n_pdbs=int(laux.rstrip('\n'))
if Start == 0:
  n_pdbs=int(laux.rstrip('\n')) - 1

os.system('rm '+InputFolder+'aux')

out_r.write('library(reticulate)\n')
out_r.write('use_python("/usr/bin/python3")\n')
out_r.write('Sys.setenv(RETICULATE_PYTHON = "/usr/bin/python3")\n')
out_r.write('reticulate::py_config()\n')
out_r.write('library(frustratometeR)\n')
out_r.write('PdbsDir <- "'+InputRoot+'"\n')
out_r.write('ResultsDir <- "'+OutputRoot+'"\n')
out_r.write('OrderList <-c()\n')
out_r.write('for(i in as.numeric('+str(Start)+'):as.numeric('+str(n_pdbs)+'))\n')
out_r.write('{OrderList <- c(OrderList, paste("'+prefix+'",i,".pdb",sep=""))}\n')
out_r.write('Dynamic_mutational <- dynamic_frustration(PdbsDir = PdbsDir, ResultsDir = ResultsDir,\n')
out_r.write('                                    GIFs = FALSE, Mode = "mutational")\n')
out_r.write('Dynamic_configurational <- dynamic_frustration(PdbsDir = PdbsDir, ResultsDir = ResultsDir,\n')
out_r.write('                                    GIFs = FALSE, Mode = "configurational")\n')
out_r.close()
print('Start frustration calculations. This might take a while (~2 frames/min)...')
os.system('udocker --allow-root run --rm -v $(pwd):/root marifrei/frustrar:latest Rscript r_frustration.R > Frustration')
print('Frustration calculations done!')

In [ ]:
#@title 5. Analysis of Per-residue Frustration using a Windowing Method
#Windowing method

l_protein=Nres
sim_start=Start

out_ref=open(InputFolder+'Reference.csv','w')
out_ref.write('Res Min Max Neu CMin CMax CNeu\n')


tam_vent=int(float(n_pdbs)*0.05) # Here we calculate the window size
ref_min=np.zeros(l_protein+1)
ref_neu=np.zeros(l_protein+1)
ref_max=np.zeros(l_protein+1)
mode=['configurational','mutational']
folder_name=InputFolder+'/pngs-all/'

os.system('mkdir '+folder_name)
residues=[]
print('Start SD and mean calculations')
for x in range(0, len(mode)):
	for i in range(sim_start, int(laux),int(tam_vent)):
		cmin=np.zeros(l_protein+1)
		cmax=np.zeros(l_protein+1)
		cneu=np.zeros(l_protein+1)
		for j in range(i, int(tam_vent)+i):
			if os.path.exists(OutputFolder+prefix+str(j)+'.done/FrustrationData/'+prefix+str(j)+'.pdb_'+mode[x]+'_5adens'):
				fst=open(OutputFolder+prefix+str(j)+'.done/FrustrationData/'+prefix+str(j)+'.pdb_'+mode[x]+'_5adens')
				for line in fst.readlines():
					if not 'nMinimallyFrst' in line:
						sp=line.rstrip('\n').split()
						cmin[int(sp[0])]+= int(sp[5]) #Adding number of Minimally frustrated contacts at 5 ams of each residue
						cmax[int(sp[0])]+= int(sp[3]) #Adding number of Highly frustrated contacts at 5 ams of each residue
						cneu[int(sp[0])]+= int(sp[4]) #Adding number of Neutral frustrated contacts at 5 ams of each residue
				fst.close()

		if i == sim_start: #if the windows y the first one, we save this parameters because we need it for the future comparations (initia window)
			for k in range(sim_start, l_protein+1):
				ref_neu[k] = cneu[k]
				ref_min[k] = cmin[k]
				ref_max[k] = cmax[k]
		else: # if te windows is not the first, we compare this with the initial window
			for k in range(1, l_protein+1):
				if ref_min[k] != 0:
					div_ref_min=(cmin[k])/ref_min[k] #here we calculate th ratio between the Wn and the W0, for the minimally frustrated residues
				else:
					div_ref_min=0
				if ref_max[k] != 0:
					div_ref_max=(cmax[k])/ref_max[k] # for highly frustrated
				else:
					div_ref_max=0
				if ref_neu[k] != 0:
					div_ref_neu=(cneu[k])/ref_neu[k] # for neutral
				else:
					div_ref_neu=0
				out_ref.write(str(k)+' '+str(div_ref_min)+' '+str(div_ref_max)+' '+str(div_ref_neu)+' '+str(cmin[k]/tam_vent)+' '+str(cmax[k]/tam_vent)+' '+str(cneu[k]/tam_vent)+'\n')

	import pandas as pd
	df = pd.read_csv(InputFolder+'Reference.csv',sep=' ')

	std_dev_grouped = df.groupby('Res')['Min'].std()
	mean_ref_grouped = df.groupby('Res')['Min'].mean()
	median_dev_grouped = df.groupby('Res')['CMin'].median()

	print("Residues identified using "+str(mode[x])+" frustration")
	residues_filtered = []
	for i in range(0, len(std_dev_grouped)):
		if mean_ref_grouped.iloc[i] != 0:
			if std_dev_grouped.iloc[i]/mean_ref_grouped.iloc[i] > 0.4 and float(median_dev_grouped.iloc[i]) > 4: #here we calculates the coefficient of variation if this number is above to 0.4 and the mean of the contacts is higher than 4 the residue change their frustration along the MD
				print(i+1,median_dev_grouped.iloc[i], std_dev_grouped.iloc[i])
				if not str((i+1)) in residues_filtered:
					residues_filtered.append(str((i+1)))

In [ ]:
#@title 6. Generating frustration plots for selected residues identified with the Windowing method
#@markdown - The plots will be available in input folder as PNG files inside a folder named "pngs-all"
#Frustration plots for selected residues using the filters:
#A script the R is generated to generate the output graphs
out_r=open('r_residues.R','w')

# Determining the number of frames
os.system('cd '+InputFolder+';ls *.pdb | wc -l > aux')
aux=open(InputFolder+'aux')
laux=aux.readline()
n_pdbs=int(laux.rstrip('\n'))
if Start == 0:
  n_pdbs=int(laux.rstrip('\n')) - 1

os.system('rm '+InputFolder+'aux')

out_r.write('library(reticulate)\n')
out_r.write('use_python("/usr/bin/python3")\n')
out_r.write('Sys.setenv(RETICULATE_PYTHON = "/usr/bin/python3")\n')
out_r.write('reticulate::py_config()\n')
out_r.write('library(frustratometeR)\n')
out_r.write('PdbsDir <- "'+InputRoot+'"\n')
out_r.write('ResultsDir <- "'+OutputRoot+'"\n')
out_r.write('OrderList <-c()\n')
out_r.write('for(i in as.numeric('+str(Start)+'):as.numeric('+str(n_pdbs)+')){OrderList <- c(OrderList, paste("'+prefix+'",i,".pdb",sep=""))}\n')
out_r.write('Dynamic_sing <- dynamic_frustration(PdbsDir = PdbsDir, ResultsDir = ResultsDir, OrderList = OrderList,\n')
out_r.write('                                    GIFs = FALSE, Mode = "singleresidue")\n')
for j in range(0,len(residues_filtered)):
   out_r.write('Dynamic_sing <- dynamic_res(Dynamic = Dynamic_sing, Resno = '+residues_filtered[j]+', Chain = "X", Graphics = TRUE)\n')
out_r.write('Dynamic_mutational <- dynamic_frustration(PdbsDir = PdbsDir, ResultsDir = ResultsDir, OrderList = OrderList,\n')
out_r.write('                                    GIFs = FALSE, Mode = "mutational")\n')
for j in range(0,len(residues_filtered)):
   out_r.write('Dynamic_mutational <- dynamic_res(Dynamic = Dynamic_mutational, Resno = '+residues_filtered[j]+', Chain = "X", Graphics = TRUE)\n')
out_r.write('Dynamic_configurational <- dynamic_frustration(PdbsDir = PdbsDir, ResultsDir = ResultsDir, OrderList = OrderList,\n')
out_r.write('                                    GIFs = FALSE, Mode = "configurational")\n')
for j in range(0,len(residues_filtered)):
   out_r.write('Dynamic_configurational <- dynamic_res(Dynamic = Dynamic_configurational, Resno = '+residues_filtered[j]+', Chain = "X", Graphics = TRUE)\n')

out_r.close()

os.system('udocker --allow-root run --rm -v $(pwd):/root marifrei/frustrar:latest Rscript r_residues.R > Results')

for j in range(0,len(residues_filtered)):
   os.system('cp '+OutputFolder+'/Dynamic_plots_res_'+residues_filtered[j]+'_X/dynamic5adens_mutational_Res'+residues_filtered[j]+'.png '+InputFolder+'/pngs-all/')
   os.system('cp '+OutputFolder+'/Dynamic_plots_res_'+residues_filtered[j]+'_X/dynamic5adens_configurational_Res'+residues_filtered[j]+'.png '+InputFolder+'/pngs-all/')
   os.system('cp '+OutputFolder+'/Dynamic_plots_res_'+residues_filtered[j]+'_X/dynamic_IndexFrustration_singleresidue_Res'+residues_filtered[j]+'.png '+InputFolder+'/pngs-all/')